# Homework 7 - Deep Learning

Обучите автоэнкодер на датасете `MNIST`, постарайтесь сделать его с размером эмбеддинга не больше 128. Сгенерируйте эмбеддинги для тысячи объектов из обучающей выборки (как в семинаре), обучите на этих эмбеддингах **Случайный лес**. Добейтесь **Accuracy** больше **90%**.

Сделайте эмбеддинги для 1000 объектов из обучающей выборки, тензор с ними должен иметь размерность (1000,*N*), где *N*≤128. Мы обучим на этом тензоре Случайный лес из библиотеки `scikit-learn`, а именно `RandomForestClassifier(random_state=0)`. После чего проверим качество на ваших эмбеддингах для тестовой выборки, сдайте их тоже. Тензор с эмбеддингами для тестовой выборки должен иметь размерность (10000,*N*). Accuracy нашего обученного случайного леса должна быть больше 90% на тестовых эмбеддингах.

Чтобы корректно сдать все эмбеддинги воспользуйтесь этой функцией:

```python
def save_embeddings(x_train, y_train, x_valid, y_valid):
    assert x_train.shape[0] == 1000
    assert x_valid.shape[0] == 10000
    
    assert y_train.shape[0] == 1000
    assert y_valid.shape[0] == 10000

    torch.save(
        {
            'x_train': x_train,
            'y_train': y_train,
            'x_valid': x_valid,
            'y_valid': y_valid
        },
        'embeddings.pt'
    )
```

## Импорт библиотек

In [3]:
import warnings

import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
from IPython.display import clear_output
from torchvision.datasets import MNIST
from PIL import Image
from matplotlib import cm
from time import perf_counter
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchvision.models import alexnet, vgg11, googlenet, resnet18
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
from typing import List, Tuple

# import importlib
# task5 = importlib.import_module("5 - Architects")
# task4 = importlib.import_module("4 - RegularizeConvNet")
# task3 = importlib.import_module("3 - ConvNets")
# task2 = importlib.import_module("2 - NeuralOptim")

%load_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore')

plt.rc('font', size=20)

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Загрузка датасета

In [4]:
mnist_transforms = T.Compose(
    [
        T.Resize((64, 64)),
        T.ToTensor(),
    ]
)

train_dataset = MNIST('mnist', train=True, transform=mnist_transforms, download=True)
valid_dataset = MNIST('mnist', train=False, transform=mnist_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 17043570.63it/s]


Extracting mnist\MNIST\raw\train-images-idx3-ubyte.gz to mnist\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 5740212.00it/s]


Extracting mnist\MNIST\raw\train-labels-idx1-ubyte.gz to mnist\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5899431.20it/s]


Extracting mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to mnist\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to mnist\MNIST\raw

